# M-Lab Real Data Collection via BigQuery

## Setup Status

✅ **Completed:**
- Google Cloud CLI installed
- Authenticated with gcloud
- Project set: `cdn-adv-comp-network-project`
- BigQuery API enabled
- BigQuery Python libraries installed

⚠️ **Required: Enable Billing**

To query public datasets like M-Lab, you need to:
1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Select your project: `cdn-adv-comp-network-project`
3. Go to **Billing** → Link a billing account (free tier available)
4. **Don't worry**: M-Lab queries are FREE up to 1 TB/month!

## Alternative: Work with Existing Data

While setting up billing, you can:
- Continue with synthetic data
- Use Lumos5G dataset (already in your workspace)
- Start RIPE Atlas measurements
- Work on feature engineering

In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path

# BigQuery imports
try:
    from google.cloud import bigquery
    import pandas_gbq
    print("✓ BigQuery libraries available")
    bigquery_available = True
except ImportError:
    print("⚠️ BigQuery not available")
    bigquery_available = False

# Set up directories
Path('../data/raw').mkdir(parents=True, exist_ok=True)
Path('../data/processed').mkdir(parents=True, exist_ok=True)

print("\n✓ Setup complete")

✓ BigQuery libraries available

✓ Setup complete


## Test BigQuery Connection

In [3]:
if bigquery_available:
    try:
        # IMPORTANT: Use measurement-lab project to query for FREE
        # Set billing_project to YOUR project (for result storage only)
        client = bigquery.Client(project='measurement-lab', 
                                default_query_job_config=bigquery.QueryJobConfig(
                                    billing_project='cdn-adv-comp-network-project'
                                ))
        print(f"✓ Connected to M-Lab project: {client.project}")
        print(f"✓ Billing to: cdn-adv-comp-network-project")
        
        # Test with a tiny query
        test_query = '''
        SELECT COUNT(*) as total
        FROM `measurement-lab.ndt.unified_downloads`
        WHERE date = '2024-11-01'
        LIMIT 1
        '''
        
        result = client.query(test_query).to_dataframe()
        print(f"✓ BigQuery working! M-Lab has {result['total'][0]:,} measurements on 2024-11-01")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 Make sure you've added measurement-lab project in BigQuery console")
else:
    print("❌ BigQuery libraries not installed")

❌ Error: Property billing_project is unknown for <class 'google.cloud.bigquery.job.query.QueryJobConfig'>.

💡 Make sure you've added measurement-lab project in BigQuery console


## M-Lab Data Query

### Query Strategy
- **Date range**: Last 30 days
- **Geographic focus**: US (start with one country)
- **Sample size**: 50,000 measurements
- **Filters**: Complete data only (no nulls)
- **Cost**: ~0.5-2 GB = FREE (within 1 TB limit)

In [ ]:
# Main M-Lab query
mlab_query = '''
SELECT
  date,
  TIMESTAMP_TRUNC(test_date, HOUR) as test_hour,
  a.MeanThroughputMbps as throughput_mbps,
  a.MinRTT as min_rtt_ms,
  a.LossRate as loss_rate,
  Client.Geo.Latitude as client_lat,
  Client.Geo.Longitude as client_lon,
  Client.Geo.City as client_city,
  Client.Geo.CountryCode as client_country,
  Client.Network.ASNumber as client_asn,
  Client.Network.ASName as client_isp,
  Server.Geo.Latitude as server_lat,
  Server.Geo.Longitude as server_lon,
  Server.Site as server_site,
  Server.Geo.City as server_city
FROM
  `measurement-lab.ndt.unified_downloads`
WHERE
  date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AND CURRENT_DATE()
  AND a.MeanThroughputMbps IS NOT NULL
  AND a.MinRTT IS NOT NULL
  AND a.MinRTT > 0
  AND a.MeanThroughputMbps > 0
  AND Client.Geo.CountryCode = 'US'
  AND Client.Geo.Latitude IS NOT NULL
  AND Server.Geo.Latitude IS NOT NULL
LIMIT 50000
'''

print("Query ready to execute:")
print(mlab_query)

In [ ]:
# Execute query (run this after billing is enabled)
if bigquery_available:
    try:
        print("Querying M-Lab dataset...")
        print("This may take 30-60 seconds...\n")
        
        client = bigquery.Client(project='cdn-adv-comp-network-project')
        df_mlab = client.query(mlab_query).to_dataframe()
        
        print(f"✓ Successfully retrieved {len(df_mlab):,} measurements!")
        print(f"\nData shape: {df_mlab.shape}")
        print(f"Date range: {df_mlab['date'].min()} to {df_mlab['date'].max()}")
        print(f"\nColumns: {list(df_mlab.columns)}")
        
        # Save raw data
        output_file = '../data/raw/mlab_ndt_us_30days.csv'
        df_mlab.to_csv(output_file, index=False)
        print(f"\n✓ Data saved to: {output_file}")
        
    except Exception as e:
        print(f"❌ Query failed: {e}")
        print("\n💡 Make sure billing is enabled in Google Cloud Console")
        df_mlab = None
else:
    print("❌ BigQuery not available")
    df_mlab = None

## Quick Data Exploration

In [ ]:
if df_mlab is not None:
    print("First few rows:")
    display(df_mlab.head(10))
    
    print("\nBasic statistics:")
    display(df_mlab[['throughput_mbps', 'min_rtt_ms', 'loss_rate']].describe())
    
    print("\nMissing values:")
    display(df_mlab.isnull().sum())

In [ ]:
if df_mlab is not None:
    # Calculate correlation
    correlation = df_mlab['min_rtt_ms'].corr(df_mlab['throughput_mbps'])
    print(f"Correlation (RTT vs Throughput): {correlation:.3f}")
    
    # Compare with synthetic data
    print("\nComparison:")
    print(f"  Real data correlation: {correlation:.3f}")
    print(f"  Synthetic data correlation: -0.666")
    print(f"\n✓ Real data validates our synthetic model!" if abs(correlation + 0.666) < 0.2 else "⚠️ Real data shows different pattern")

## Visualization: Real vs Synthetic Data

In [ ]:
if df_mlab is not None:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Real data scatter
    axes[0].scatter(df_mlab['min_rtt_ms'], df_mlab['throughput_mbps'], 
                    alpha=0.3, s=10)
    axes[0].set_xlabel('Minimum RTT (ms)')
    axes[0].set_ylabel('Throughput (Mbps)')
    axes[0].set_title(f'Real M-Lab Data\n(n={len(df_mlab):,}, corr={correlation:.3f})')
    axes[0].grid(True, alpha=0.3)
    
    # Load synthetic for comparison
    df_synthetic = pd.read_csv('../data/processed/mlab_synthetic_sample.csv')
    corr_synthetic = df_synthetic['min_rtt_ms'].corr(df_synthetic['throughput_mbps'])
    
    axes[1].scatter(df_synthetic['min_rtt_ms'], df_synthetic['throughput_mbps'],
                    alpha=0.3, s=10, color='orange')
    axes[1].set_xlabel('Minimum RTT (ms)')
    axes[1].set_ylabel('Throughput (Mbps)')
    axes[1].set_title(f'Synthetic Data\n(n={len(df_synthetic):,}, corr={corr_synthetic:.3f})')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/figures/real_vs_synthetic_comparison.png', dpi=300, bbox_inches='tight')
    print("✓ Comparison plot saved")
    plt.show()

## Alternative: Use Lumos5G Dataset

While waiting for BigQuery billing, you can explore the Lumos5G dataset already in your workspace!

In [ ]:
# Load Lumos5G dataset
lumos_file = '../../Lumos5G-v1.0/Lumos5G-v1.0.csv'

if os.path.exists(lumos_file):
    print("Loading Lumos5G dataset...")
    df_lumos = pd.read_csv(lumos_file)
    print(f"✓ Loaded {len(df_lumos):,} measurements from Lumos5G")
    print(f"\nColumns: {list(df_lumos.columns)}")
    print("\nFirst few rows:")
    display(df_lumos.head())
else:
    print(f"❌ Lumos5G file not found: {lumos_file}")

## Next Steps

### If BigQuery is Working:
1. ✅ You now have 50,000 real measurements
2. Compare real vs synthetic correlation
3. Move to Phase 5: Feature Engineering

### If Waiting for Billing:
1. Continue with synthetic data
2. Explore Lumos5G dataset
3. Start RIPE Atlas measurements (Phase 4)
4. Work on ML models with synthetic data

### Resources:
- [Enable Billing](https://console.cloud.google.com/billing)
- [BigQuery Free Tier](https://cloud.google.com/bigquery/pricing#free-tier)
- [M-Lab Documentation](https://www.measurementlab.net/data/)